# 🧹 Ayudantía 5: Transformación y Limpieza de Datos en Pandas

**Objetivos**:
- Detectar y **eliminar** valores faltantes (sin imputar).
- Normalizar strings y columnas semi‑estructuradas (`split`, `explode`).
- Convertir tipos (`astype`, `to_datetime`) sin rellenar.
- Conectar DataFrames (`merge`, `concat`, `join`) y analizar cardinalidad.


In [ ]:
import pandas as pd
import numpy as np

## 1) Dataset de ejemplo
Contiene problemas comunes: espacios, mayúsculas/acentos, listas en una sola celda, fechas mixtas, montos con símbolos.


In [3]:
df = pd.DataFrame({
    'id': [' 001','002','003','004','003 '],
    'nombre_completo': ['ana GÓmez','  LUIS  pérez ','Sofía  Díaz', None,'sofia  DÍAZ'],
    'emails': ['ana@uc.cl; ana@gmail.com', 'l.perez@uc.cl', 'sdiaz@uc.cl;SOFIA@MAIL.COM', 'pedro@uc.cl', None],
    'fecha_atencion': ['12/03/2024','31-04-2024','2024-05-10','2024-06-15','10-05-24'],
    'monto': ['10000', '12.500', '4.500,00', '15.750$', 'USD 3,000.50'],
    'fono': ['123456789', '9-8765-4321', '(+56) 9 1111 2222', '987654321', 'fono: 456789123']
})

df

,id,nombre_completo,emails,fecha_atencion,monto,fono
0,001,ana GÓmez,ana@uc.cl; ana@gmail.com,12/03/2024,10000,123456789
1,002,LUIS pérez,l.perez@uc.cl,31-04-2024,12.500,9-8765-4321
2,003,Sofía Díaz,sdiaz@uc.cl;SOFIA@MAIL.COM,2024-05-10,"4.500,00",(+56) 9 1111 2222
3,004,None,pedro@uc.cl,2024-06-15,15.750$,987654321
4,003,sofia DÍAZ,None,10-05-24,"USD 3,000.50",fono: 456789123


## 2) Limpieza
Pasos: normalizar `id`, quitar nulos y duplicados, separar nombre/apellido, explotar correos, parsear fechas (errores→NaT), convertir monto a numérico, unir con correos.


In [4]:
df.info() # Información general del DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               5 non-null      object
 1   nombre_completo  4 non-null      object
 2   emails           4 non-null      object
 3   fecha_atencion   5 non-null      object
 4   monto            5 non-null      object
 5   fono             5 non-null      object
dtypes: object(6)
memory usage: 372.0+ bytes


In [5]:
df.isna().sum() # Conteo de valores nulos por columna

id                 0
nombre_completo    1
emails             1
fecha_atencion     0
monto              0
fono               0
dtype: int64

In [6]:
df[df['emails'].isna()]        # Filas donde 'col' es NaN

,id,nombre_completo,emails,fecha_atencion,monto,fono
4,003,sofia DÍAZ,None,10-05-24,"USD 3,000.50",fono: 456789123


In [7]:
df =df.dropna(subset=['nombre_completo'])  # Elimina filas con NaN en 'nombre_completo'
df

,id,nombre_completo,emails,fecha_atencion,monto,fono
0,001,ana GÓmez,ana@uc.cl; ana@gmail.com,12/03/2024,10000,123456789
1,002,LUIS pérez,l.perez@uc.cl,31-04-2024,12.500,9-8765-4321
2,003,Sofía Díaz,sdiaz@uc.cl;SOFIA@MAIL.COM,2024-05-10,"4.500,00",(+56) 9 1111 2222
4,003,sofia DÍAZ,None,10-05-24,"USD 3,000.50",fono: 456789123


In [8]:
df.reset_index(drop=True, inplace=True)  # Reinicia el índice después de eliminar filas
df

,id,nombre_completo,emails,fecha_atencion,monto,fono
0,001,ana GÓmez,ana@uc.cl; ana@gmail.com,12/03/2024,10000,123456789
1,002,LUIS pérez,l.perez@uc.cl,31-04-2024,12.500,9-8765-4321
2,003,Sofía Díaz,sdiaz@uc.cl;SOFIA@MAIL.COM,2024-05-10,"4.500,00",(+56) 9 1111 2222
3,003,sofia DÍAZ,None,10-05-24,"USD 3,000.50",fono: 456789123


In [9]:
# Reemplazos comunes
df['fono'] = (df['fono']
              .str.replace(r'\D+', '', regex=True)  # Dejar solo dígitos
              .str.replace(r'^56', '', regex=True))  # Quitar código país si viene repetido
df

C:\Users\djop4\AppData\Local\Temp\ipykernel_10844\1142249288.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fono'] = (df['fono']


,id,nombre_completo,emails,fecha_atencion,monto,fono
0,001,ana GÓmez,ana@uc.cl; ana@gmail.com,12/03/2024,10000,123456789
1,002,LUIS pérez,l.perez@uc.cl,31-04-2024,12.500,987654321
2,003,Sofía Díaz,sdiaz@uc.cl;SOFIA@MAIL.COM,2024-05-10,"4.500,00",911112222
3,003,sofia DÍAZ,None,10-05-24,"USD 3,000.50",456789123


In [10]:
clean = df.copy()

# 1) Normalizar ID: quitar espacios y ceros a la izquierda → Int64 (permite NA)
clean['id'] = (clean['id'].str.strip()
                         .str.replace(r'^0+', '', regex=True)
                         .replace('', None)
                         .astype('Int64'))

clean

,id,nombre_completo,emails,fecha_atencion,monto,fono
0,1,ana GÓmez,ana@uc.cl; ana@gmail.com,12/03/2024,10000,123456789
1,2,LUIS pérez,l.perez@uc.cl,31-04-2024,12.500,987654321
2,3,Sofía Díaz,sdiaz@uc.cl;SOFIA@MAIL.COM,2024-05-10,"4.500,00",911112222
3,3,sofia DÍAZ,None,10-05-24,"USD 3,000.50",456789123


In [11]:
# 2) Normalizar nombre (Title Case y colapso de espacios)
clean['nombre_completo'] = clean['nombre_completo'].str.strip().str.title().str.replace(r'\s+', ' ', regex=True)
clean[['nombre','apellido']] = clean['nombre_completo'].str.split(' ', n=1, expand=True)

clean

,id,nombre_completo,emails,fecha_atencion,monto,fono,nombre,apellido
0,1,Ana Gómez,ana@uc.cl; ana@gmail.com,12/03/2024,10000,123456789,Ana,Gómez
1,2,Luis Pérez,l.perez@uc.cl,31-04-2024,12.500,987654321,Luis,Pérez
2,3,Sofía Díaz,sdiaz@uc.cl;SOFIA@MAIL.COM,2024-05-10,"4.500,00",911112222,Sofía,Díaz
3,3,Sofia Díaz,None,10-05-24,"USD 3,000.50",456789123,Sofia,Díaz


In [12]:
# 3) Correos a filas (explode)

new = clean.copy()
new['emails_list'] = new['emails'].str.lower().str.replace(' ', '', regex=False).str.split(';')
new = new.explode('emails_list', ignore_index=True)
new = new.rename(columns={'emails_list':'email'})
# No eliminamos los vacíos, así se mantienen las filas sin email
new

,id,nombre_completo,emails,fecha_atencion,monto,fono,nombre,apellido,email
0,1,Ana Gómez,ana@uc.cl; ana@gmail.com,12/03/2024,10000,123456789,Ana,Gómez,ana@uc.cl
1,1,Ana Gómez,ana@uc.cl; ana@gmail.com,12/03/2024,10000,123456789,Ana,Gómez,ana@gmail.com
2,2,Luis Pérez,l.perez@uc.cl,31-04-2024,12.500,987654321,Luis,Pérez,l.perez@uc.cl
3,3,Sofía Díaz,sdiaz@uc.cl;SOFIA@MAIL.COM,2024-05-10,"4.500,00",911112222,Sofía,Díaz,sdiaz@uc.cl
4,3,Sofía Díaz,sdiaz@uc.cl;SOFIA@MAIL.COM,2024-05-10,"4.500,00",911112222,Sofía,Díaz,sofia@mail.com
5,3,Sofia Díaz,None,10-05-24,"USD 3,000.50",456789123,Sofia,Díaz,None


In [ ]:
new = new.drop(columns=['emails'])
new

,id,nombre_completo,fecha_atencion,monto,fono,nombre,apellido,email
0,1,Ana Gómez,12/03/2024,10000,123456789,Ana,Gómez,ana@uc.cl
1,1,Ana Gómez,12/03/2024,10000,123456789,Ana,Gómez,ana@gmail.com
2,2,Luis Pérez,31-04-2024,12.500,987654321,Luis,Pérez,l.perez@uc.cl
3,3,Sofía Díaz,2024-05-10,"4.500,00",911112222,Sofía,Díaz,sdiaz@uc.cl
4,3,Sofía Díaz,2024-05-10,"4.500,00",911112222,Sofía,Díaz,sofia@mail.com
5,3,Sofia Díaz,10-05-24,"USD 3,000.50",456789123,Sofia,Díaz,None


In [14]:
# 4) Fechas a datetime (sin imputar; errores->NaT)


## USANDO TO_DATATIME
new_1 = new.copy()

# Supongamos que tu DF se llama df
col = new_1['fecha_atencion'].astype(str)

# Lista de formatos que queremos intentar
formatos = ["%d/%m/%Y", "%d-%m-%Y", "%Y-%m-%d", "%d-%m-%y"]

# Lista para guardar los resultados parciales
parsed_list = []

for fmt in formatos:
    parsed = pd.to_datetime(col, format=fmt, errors="coerce")
    parsed_list.append(parsed)

# Concatenar todos los resultados
df_concat = pd.concat(parsed_list, axis=1)

# Tomar el primer valor no nulo por fila
new_1['fecha_atencion'] = df_concat.bfill(axis=1).iloc[:, 0]

# Normalizar el formato
new_1['fecha_atencion'] = new_1['fecha_atencion'].dt.strftime("%Y-%m-%d")
new_1


,id,nombre_completo,fecha_atencion,monto,fono,nombre,apellido,email
0,1,Ana Gómez,2024-03-12,10000,123456789,Ana,Gómez,ana@uc.cl
1,1,Ana Gómez,2024-03-12,10000,123456789,Ana,Gómez,ana@gmail.com
2,2,Luis Pérez,NaN,12.500,987654321,Luis,Pérez,l.perez@uc.cl
3,3,Sofía Díaz,2024-05-10,"4.500,00",911112222,Sofía,Díaz,sdiaz@uc.cl
4,3,Sofía Díaz,2024-05-10,"4.500,00",911112222,Sofía,Díaz,sofia@mail.com
5,3,Sofia Díaz,2024-05-10,"USD 3,000.50",456789123,Sofia,Díaz,None


In [15]:
## USANDO UNA FUNCIÓN PERSONALIZADA
from datetime import datetime

new_2 = new.copy()

def parse_fecha(fecha):
    formatos = ["%d/%m/%Y", "%d-%m-%Y", "%Y-%m-%d", "%d-%m-%y"]
    for fmt in formatos:
        try:
            return datetime.strptime(str(fecha), fmt)
        except ValueError:
            continue
    return pd.NaT  # si no calza con ninguno

new_2['fecha_atencion'] = new_2['fecha_atencion'].apply(parse_fecha)

# Normalizar a YYYY-MM-DD
new_2['fecha_atencion'] = new_2['fecha_atencion'].dt.strftime("%Y-%m-%d")

new_2


,id,nombre_completo,fecha_atencion,monto,fono,nombre,apellido,email
0,1,Ana Gómez,2024-03-12,10000,123456789,Ana,Gómez,ana@uc.cl
1,1,Ana Gómez,2024-03-12,10000,123456789,Ana,Gómez,ana@gmail.com
2,2,Luis Pérez,NaN,12.500,987654321,Luis,Pérez,l.perez@uc.cl
3,3,Sofía Díaz,2024-05-10,"4.500,00",911112222,Sofía,Díaz,sdiaz@uc.cl
4,3,Sofía Díaz,2024-05-10,"4.500,00",911112222,Sofía,Díaz,sofia@mail.com
5,3,Sofia Díaz,2024-05-10,"USD 3,000.50",456789123,Sofia,Díaz,None


In [16]:
# 5) Monto a numérico
# Conserva comas y puntos, elimina otros símbolos (letras, $ y espacios)
def monto_simple(valor):
    if pd.isna(valor):
        return None
    s = str(valor)
    
    # dejar solo dígitos, comas y puntos con replace
    for ch in s:
        if not (ch.isdigit() or ch in [',','.']):
            s = s.replace(ch, '')
    
    # si hay más de un separador -> el último es decimal
    if s.count('.') + s.count(',') > 1:
        # buscamos el último separador
        idx = max(s.rfind('.'), s.rfind(','))
        # quitamos todos los separadores anteriores
        s = s[:idx].replace('.', '').replace(',', '') + '.' + s[idx+1:].replace(',', '').replace('.', '')
    else:
        # si hay solo coma, la usamos como decimal
        s = s.replace(',', '.')
    
    try:
        return float(s)
    except:
        return None

new_1['monto'] = new_1['monto'].apply(monto_simple)

final = new_1.copy()
final

,id,nombre_completo,fecha_atencion,monto,fono,nombre,apellido,email
0,1,Ana Gómez,2024-03-12,10000.0,123456789,Ana,Gómez,ana@uc.cl
1,1,Ana Gómez,2024-03-12,10000.0,123456789,Ana,Gómez,ana@gmail.com
2,2,Luis Pérez,NaN,12.5,987654321,Luis,Pérez,l.perez@uc.cl
3,3,Sofía Díaz,2024-05-10,4500.0,911112222,Sofía,Díaz,sdiaz@uc.cl
4,3,Sofía Díaz,2024-05-10,4500.0,911112222,Sofía,Díaz,sofia@mail.com
5,3,Sofia Díaz,2024-05-10,3000.5,456789123,Sofia,Díaz,None


## 3) Conexión de DataFrames y cardinalidad
Ejemplo: pacientes y atenciones.


In [17]:
pac = pd.DataFrame({'id':[1,2,3], 'nombre':['Ana','Luis','Sofía']})
att = pd.DataFrame({'id':[1,1,2,4], 'fecha':['2024-05-01','2024-05-12','2024-06-05','2024-07-01']})
att['fecha'] = pd.to_datetime(att['fecha'])

m_left = pd.merge(pac, att, on='id', how='left')
m_inner = pd.merge(pac, att, on='id', how='inner')
m_outer = pd.merge(pac, att, on='id', how='outer')

m_left

,id,nombre,fecha
0,1,Ana,2024-05-01
1,1,Ana,2024-05-12
2,2,Luis,2024-06-05
3,3,Sofía,NaT


In [18]:
m_inner

,id,nombre,fecha
0,1,Ana,2024-05-01
1,1,Ana,2024-05-12
2,2,Luis,2024-06-05


In [19]:
m_outer

,id,nombre,fecha
0,1,Ana,2024-05-01
1,1,Ana,2024-05-12
2,2,Luis,2024-06-05
3,3,Sofía,NaT
4,4,NaN,2024-07-01
